# Gender : Female
# Name : Cadeton Precious
# School : Ambrose Alli University
# AI club : AI club AAU chapter

In [ ]:
pip install git+https://github.com/MaxHalford/xam --upgrade

In [ ]:
#Importation of some python libraries needed
import pandas as  pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xam

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#Read in csv files
train = pd.read_csv('/kaggle/input/intercampusai2019/train.csv')
test = pd.read_csv('/kaggle/input/intercampusai2019/test.csv')
hsample = pd.read_csv('/kaggle/input/intercampusai2019/sample_submission2.csv')

test_id = test['EmployeeNo']

In [ ]:
#Getting the length of the train data
len_train = len(train)

In [ ]:
train.drop(columns='EmployeeNo', inplace=True)
test.drop(columns='EmployeeNo', inplace=True)

In [ ]:
#Feature Engineering, it is used to get categorical columns out of train data

In [ ]:
#Assigned all categorical columns to a variable
cat_col = list(train.select_dtypes(include='object').columns)
print(cat_col)

In [ ]:
train.head()

In [ ]:
#Getting the actual age of the employees
train['Year_of_birth'] = 2019 - train['Year_of_birth']
test['Year_of_birth'] = 2019 - test['Year_of_birth']

train['Year_of_recruitment'] = 2019 - train['Year_of_recruitment']
test['Year_of_recruitment'] = 2019 - test['Year_of_recruitment']

In [ ]:
train.head()

In [ ]:
t_cluster = train[['Year_of_birth', 'Year_of_recruitment']]
test_cluster = test[['Year_of_birth', 'Year_of_recruitment']]

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
k_train = KMeans(n_clusters = 5)
k_train.fit(t_cluster)
train['Cluster'] = k_train.predict(t_cluster)

k_test = KMeans(n_clusters = 5)
k_test.fit(test_cluster)
test['Cluster'] = k_test.predict(test_cluster)

In [ ]:
train.head()

In [ ]:
#Checking for missing values
train.isna().sum()

In [ ]:
#Missing values filling, instead of deleting it 
train['Qualification'].fillna(value='First Degree or HND',inplace=True)
test['Qualification'].fillna(value='First Degree or HND',inplace=True)

In [ ]:
mean_ecode = xam.feature_extraction.BayesianTargetEncoder(columns=cat_col)
y_target = train['Promoted_or_Not']
train_ecode = train.drop('Promoted_or_Not', axis=1)
mean_ecode.fit(train_ecode, y_target)

train = mean_ecode.transform(train_ecode)
test = mean_ecode.transform(test)

In [ ]:
train.drop(cat_col, axis=1, inplace=True)
test.drop(cat_col, axis=1, inplace=True)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Missing values filling
X_train,X_test,y_train,y_test = train_test_split(train,y_target, test_size = 0.3, random_state = 0)

In [ ]:
#Missing values filling
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score

In [ ]:
import catboost as cb

In [ ]:
catb = cb.CatBoostClassifier(iterations=1000, use_best_model=True, eval_metric='F1', random_seed=1)
catb.fit(X_train, y_train, eval_set = (X_test, y_test))

In [ ]:
cb_model = cb.CatBoostClassifier(iterations=762, random_seed=1, silent=True)
cb_model.fit(X_train, y_train)
cb_pred = cb_model.predict(X_test)
f1_score(cb_pred, y_test)

In [ ]:
gb = GradientBoostingClassifier(n_estimators=300, random_state=2)
gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)

In [ ]:
f1_score(gb_pred, y_test)

In [ ]:
st =StandardScaler()
st.fit(train)
train_st = st.transform(train)
test_st = st.transform(test)

In [ ]:
cb_model.fit(train_st, y_target)
final_pred = cb_model.predict(test_st)

In [ ]:
#for my final prediction i decided to use catboost beacuase it improved my model

In [ ]:
sample.head()

In [ ]:
sample['Promoted_or_Not'] = final_pred
sample['EmployeeNo'] = test_id

In [ ]:
#converting Promoted_or_Not to float because it came in same form

In [ ]:
sample['Promoted_or_Not'] = sample["Promoted_or_Not"].astype("int64")

In [ ]:
sample.Promoted_or_Not.value_counts()

In [ ]:
sample.head()

In [ ]:
sample.to_csv("Best_Model.csv", index=False)